### Exercise 1-4.

 
Here is a similar profit-maximizing model, but in a different context. An automobile manufacturer produces several kinds of cars. Each kind requires a certain amount of factory time per car to produce, and yields a certain profit per car. A certain amount of factory time has been scheduled for the next week, and it is desired to use all this time; but at least a certain number of each kind of car must be manufactured to meet dealer requirements.

(a) What are the data values that define this problem? How would you declare the sets and parameter values for this problem in AMPL? What are the decision variables, and how would you declare them in AMPL?

(b) Assuming that the objective is to maximize total profit, how would you declare an objective in AMPL for this problem? How would you declare the constraints?

(c) For purposes of experiment, suppose that there are three kinds of cars, known at the factory as T, C and L, that 120 hours are available, and that the time per car, profit per car and dealer orders for each kind of car are as follows:
Car time profit orders

|Car |time_per_hour|profit_per_car|min_order_per_car|
|:---|             |              |             ---:|
|T   |            1|           200|             10  |
|C   |            2|           500|             20  |
|L   |            3|           700|             15  |

How much of each car should be produced, and what is the maximum profit? You should find that your solution specifies a fractional amount of one of the cars. As a practical matter, how could you make use of this solution?

(d) If you maximize the total number of cars produced instead of the total profit, how many more cars do you make? How much less profit?

(e) Each kind of car achieves a certain fuel efficiency, and the manufacturer is required by law to maintain a certain ‘‘fleet average’’ efficiency. The fleet average is computed by multiplying the efficiency of each kind of car times the number of that kind produced, summing all of the resulting products, and dividing by the total of all cars produced. Extend your AMPL model to contain a minimum fleet average efficiency constraint. Rearrange the constraint as necessary to make it linear — no variables divided into other variables.

(f) Find the optimal solution for the case where cars T, C and L achieve fuel efficiencies of 50, 30 and 20 miles/gallon, and the fleet average efficiency must be at least 35 miles/gallon. Explain how this changes the production amounts and the total profit. Dealing with the fractional amounts in the solution is not so easy in this case. What might you do? If you had 10 more hours of production time, you could make more profit. Does the addition of the fleet average efficiency constraint make the extra 10 hours more or less valuable?

(g) Explain how you could further refine this model to account for different production stages that have different numbers of hours available per stage, much as in the steel model of Section 1.6.

In [4]:
import pandas as pd
from amplpy import AMPL, Environment

In [5]:
ampl = AMPL(Environment('/opt/ampl.linux64'))
ampl.reset()

In [6]:
ampl.read('./ex--1-4abc.mod')

In [7]:
ampl.readData('./ex--1-4abc.dat')

In [8]:
variables = ('T','C','L')
constraint_names = ('time_per_car')

In [10]:
def model_overview(ampl=None):
    print('Variables and their bounds')
    for i in ampl.getVariables():
        [print(i[1].get(j)) for j in variables]
    print()
    print('Constraints and their limits')
    print(ampl.getConstraint('constraints'))
    print()
    print('Constraints and their limits. Alternate way to print.')
    print(ampl.getConstraint('constraints').get('time_per_car'))
    print()
    for i in ampl.getConstraints():
        print(i[1].get('time_per_car'))
    print()
    print('Objective Function')
    print(ampl.getObjective('total_profit').get())

In [11]:
model_overview(ampl=ampl)

Variables and their bounds
var X['T'] >=10;
var X['C'] >=20;
var X['L'] >=15;

Constraints and their limits
subject to constraints{c in constraint_names} : sum{i in cars}
  constraint_list[i,c]*X[i] <= constraint_limits[c];

Constraints and their limits. Alternate way to print.
subject to constraints['time_per_car']:
	X['T'] + 2*X['C'] + 3*X['L'] <= 120;

subject to constraints['time_per_car']:
	X['T'] + 2*X['C'] + 3*X['L'] <= 120;

Objective Function
maximize total_profit:
	200*X['T'] + 500*X['C'] + 700*X['L'];


In [12]:
ampl.eval('expand total_profit;')

maximize total_profit:
	200*X['T'] + 500*X['C'] + 700*X['L'];



In [13]:
ampl.eval('expand constraints;')

subject to constraints['time_per_car']:
	X['T'] + 2*X['C'] + 3*X['L'] <= 120;



In [14]:
for variable in variables:
    ampl.eval('expand X["{}"];'.format(variable))

Coefficients of X['T']:
	constraints['time_per_car']    1
	total_profit                 200

Coefficients of X['C']:
	constraints['time_per_car']    2
	total_profit                 500

Coefficients of X['L']:
	constraints['time_per_car']    3
	total_profit                 700



In [15]:
ampl.eval('display _nvars, _ncons;')

_nvars = 3
_ncons = 1



In [16]:
print(ampl.getOption('solver'))
ampl.setOption('solver','cplex')
print(ampl.getOption('solver'))

cplex
cplex


In [17]:
ampl.solve()
print(ampl.getObjective('total_profit').get().value())

CPLEX 12.8.0.0: optimal solution; objective 28750
0 simplex iterations (0 in phase I)
28750.0


#### Shadow Price or Dual Price or Marginal Price

In [18]:
ampl.eval('display total_profit;')
for i in ampl.getConstraints():
    print(i[1].getValues())
#print(ampl.getConstraint('Time').getValues())

total_profit = 28750

   index0    | constraints.dual
'time_per_car' |     250     



### Decision Variable Solution and Reduced Cost

In [20]:
X_result = list()
for p in variables:
    X_result.append([
        p,
        ampl.getVariable('X').get(p).lb(),
        ampl.getVariable('X').get(p).value(),
        ampl.getVariable('X').get(p).ub(),
        ampl.getVariable('X').get(p).rc()
    ])
df_X_result = pd.DataFrame(X_result)
df_X_result.rename(columns={0:'DecisionVariable_X',
                            1:'X_LowerBound',
                            2:'X_Solution',
                            3:'X_UpperBound',
                            4:'X_ReducedCosts'},
                   inplace=True)
df_X_result.set_index('DecisionVariable_X',inplace=True)
df_X_result

,X_LowerBound,X_Solution,X_UpperBound,X_ReducedCosts
DecisionVariable_X,,,,
T,10.0,10.0,inf,-50.0
C,20.0,32.5,inf,0.0
L,15.0,15.0,inf,-50.0


In [21]:
print(ampl.getData('X'))
values = ampl.getVariable('X').getValues()
print(values)
df = values.toPandas()
print(values.toList())
print(values.toDict())

   index0    |      X      
    'C'      |     32.5    
    'L'      |      15     
    'T'      |      10     

   index0    |    X.val    
    'C'      |     32.5    
    'L'      |      15     
    'T'      |      10     

[('C', 32.5), ('L', 15.0), ('T', 10.0)]
{'C': 32.5, 'L': 15.0, 'T': 10.0}


In [22]:
import numpy as np
df_result = df_X_result.join(ampl.getEntity('profit_per_car').getValues().toPandas())
df_result['X_Solution.floor'] = df_result['X_Solution'].apply(lambda x: np.floor(x))
df_result

,X_LowerBound,X_Solution,X_UpperBound,X_ReducedCosts,profit_per_car,X_Solution.floor
DecisionVariable_X,,,,,,
T,10.0,10.0,inf,-50.0,200.0,10.0
C,20.0,32.5,inf,0.0,500.0,32.0
L,15.0,15.0,inf,-50.0,700.0,15.0


In [23]:
total_profit = (df_result['X_Solution'] * df_result['profit_per_car']).sum()
total_adj_profit = (df_result['X_Solution.floor'] * df_result['profit_per_car']).sum()
diff = total_profit - total_adj_profit
print('total profit: ${}'.format(total_profit))
print('total adjusted profit: ${}'.format(total_adj_profit))
print('diff in profit: ${}'.format(diff))

total profit: $28750.0
total adjusted profit: $28500.0
diff in profit: $250.0
